In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
smile = pd.read_csv('../data/smile_clean.csv').drop('Unnamed: 0', axis = 1)
smile.head(3)

In [ ]:
nash = pd.read_csv('../data/nash_visitors.csv').drop('Unnamed: 0', axis = 1)
nash.head(3)

In [ ]:
memp = pd.read_csv('../data/memp_visitors.csv').drop('Unnamed: 0', axis = 1)
memp.head(3)

In [ ]:
upper = smile.value_counts("num of upper natural teeth")

In [ ]:
lower = smile.value_counts("num of lower natural teeth")

### trying to utilize DP's code for dental issues

In [ ]:
smile.loc[~smile['callernum'].isin([-1,-2])]['callernum'].nunique()

In [ ]:
clients_list = smile.loc[~smile['callernum'].isin([-1,-2])]['callernum'].nunique()
clients_list